# Example how to read text embeddings



In [1]:
from src.vectordb import VectorDB
from src.embeddings import Embedder

# Enable autoreload for development
%load_ext autoreload
%autoreload 2

In [2]:

emb = Embedder("michiyasunaga/BioLinkBERT-large", "cpu")
vecdb = VectorDB("../vectors.chromadb", embedder=emb)

vecs = vecdb.dump_vectors_with_mesh_terms()

Running embedder on device: cpu


In [3]:
vecs



[{'pubmed_id': 36291189,
  'segment': 0,
  'embedding': array([-0.17757398,  0.02531927,  0.23698409, ...,  0.14285968,
         -0.03779435,  0.02688294], shape=(1024,)),
  'mesh_terms': 'Mesenchymal Stem Cells,Humans,NF-kappa B,RNA,Small Interfering,Interleukin-8,Osteogenesis'},
 {'pubmed_id': 36291189,
  'segment': 1,
  'embedding': array([-0.21293205, -0.04633248,  0.43611354, ...,  0.10857925,
         -0.13227884, -0.06309651], shape=(1024,)),
  'mesh_terms': 'Mesenchymal Stem Cells,Humans,NF-kappa B,RNA,Small Interfering,Interleukin-8,Osteogenesis'},
 {'pubmed_id': 36291189,
  'segment': 2,
  'embedding': array([-0.21493745,  0.13799019,  0.35579705, ...,  0.12439117,
         -0.02336557, -0.04290207], shape=(1024,)),
  'mesh_terms': 'Mesenchymal Stem Cells,Humans,NF-kappa B,RNA,Small Interfering,Interleukin-8,Osteogenesis'},
 {'pubmed_id': 36291189,
  'segment': 3,
  'embedding': array([-0.26763701,  0.15418902,  0.36699203, ...,  0.11718009,
         -0.04576641,  0.03252376]

## Save Text Embeddings with Metadata

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime

# Extract embeddings and metadata
embeddings_list = []
metadata_list = []

for idx, item in enumerate(vecs):
    embeddings_list.append(item['embedding'])
    
    # Create text_id in format: pubmed_<pubmedid>_seg<segment>
    text_id = f"pubmed_{item['pubmed_id']}_seg{item['segment']}"
    
    metadata_list.append({
        'text_id': text_id,
        'pubmed_id': item['pubmed_id'],
        'segment': item['segment'],
        'mesh_terms': item['mesh_terms'],
        'embedding_index': idx
    })

# Convert to numpy array
text_embeddings = np.array(embeddings_list)
print(f"Text embeddings shape: {text_embeddings.shape}")

# Create metadata DataFrame with ordered columns
metadata_df = pd.DataFrame(metadata_list)
# Reorder columns to match cell embeddings format
metadata_df = metadata_df[['text_id', 'pubmed_id', 'segment', 'mesh_terms', 'embedding_index']]

print(f"Metadata rows: {len(metadata_df)}")
print(f"\nFirst 5 rows:")
print(metadata_df.head())

Text embeddings shape: (17633, 1024)
Metadata rows: 17633

First 5 rows:
                text_id  pubmed_id  segment  \
0  pubmed_36291189_seg0   36291189        0   
1  pubmed_36291189_seg1   36291189        1   
2  pubmed_36291189_seg2   36291189        2   
3  pubmed_36291189_seg3   36291189        3   
4  pubmed_36291189_seg4   36291189        4   

                                          mesh_terms  embedding_index  
0  Mesenchymal Stem Cells,Humans,NF-kappa B,RNA,S...                0  
1  Mesenchymal Stem Cells,Humans,NF-kappa B,RNA,S...                1  
2  Mesenchymal Stem Cells,Humans,NF-kappa B,RNA,S...                2  
3  Mesenchymal Stem Cells,Humans,NF-kappa B,RNA,S...                3  
4  Mesenchymal Stem Cells,Humans,NF-kappa B,RNA,S...                4  


In [5]:
# Save to files
output_folder = "/Users/vedran/Documents/scpgpt-project/text-embeddings"  # or wherever you want to save
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save embeddings as numpy array
embeddings_file = f"{output_folder}/text_embeddings_{timestamp}.npy"
np.save(embeddings_file, text_embeddings)
print(f"✓ Saved embeddings: {embeddings_file}")

# Save metadata as CSV
metadata_csv = f"{output_folder}/text_metadata_{timestamp}.csv"
metadata_df.to_csv(metadata_csv, index=False)
print(f"✓ Saved metadata: {metadata_csv}")

print(f"\n📁 Files saved:")
print(f"  1. Embeddings (NPY): {embeddings_file}")
print(f"  2. Metadata (CSV):   {metadata_csv}")
print(f"\nTo load later:")
print(f"  embeddings = np.load('{embeddings_file}')")
print(f"  metadata = pd.read_csv('{metadata_csv}')")
print(f"\nUse 'embedding_index' to match embeddings with metadata rows.")

✓ Saved embeddings: /Users/vedran/Documents/scpgpt-project/text-embeddings/text_embeddings_20251106_165002.npy
✓ Saved metadata: /Users/vedran/Documents/scpgpt-project/text-embeddings/text_metadata_20251106_165002.csv

📁 Files saved:
  1. Embeddings (NPY): /Users/vedran/Documents/scpgpt-project/text-embeddings/text_embeddings_20251106_165002.npy
  2. Metadata (CSV):   /Users/vedran/Documents/scpgpt-project/text-embeddings/text_metadata_20251106_165002.csv

To load later:
  embeddings = np.load('/Users/vedran/Documents/scpgpt-project/text-embeddings/text_embeddings_20251106_165002.npy')
  metadata = pd.read_csv('/Users/vedran/Documents/scpgpt-project/text-embeddings/text_metadata_20251106_165002.csv')

Use 'embedding_index' to match embeddings with metadata rows.
